In [1]:
# ! pip install optuna
# ! pip install torch
# ! pip install pandas
# ! pip install numpy
# ! pip install matplotlib
# ! pip install kagglehub
# ! pip install scikit-learn

In [2]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

import optuna

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# path = "/kaggle/input/fashionmnist"

import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fashionmnist


In [5]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
sample_df = df.sample(n=60000, random_state=42)

In [7]:
X = sample_df.iloc[:, 1:].values
y = sample_df.iloc[:, :1].values


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
st_scaler = StandardScaler()

In [10]:

X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

# Dataset Class

In [11]:
# create custom dataset class

class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels):
        
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28) # converted 1D 784 rows into 28x28 2D rows, 
                                                                                        # 1 states that its a grayscale image, -1 is the batch size
        self.labels = torch.tensor(labels, dtype=torch.long)
        
        


    def __len__(self):
        return len(self.features)


    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [12]:
train_dataset = FashionMNISTDataset(X_train, y_train)
test_dataset = FashionMNISTDataset(X_test, y_test)

In [13]:
# loading data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)


In [14]:
class CNNModel(nn.Module):
    
    def __init__(self, input_features):
        
        super().__init__()
        
        # convolutions and pooling layer
        self.features = nn.Sequential(
            nn.Conv2d(
                in_channels = input_features,
                out_channels = 32,
                kernel_size = 3,
                padding = 'same' 
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(
                in_channels = 32,
                out_channels = 64,
                kernel_size = 3,
                padding = 'same' 
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7 , 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(64, 10)
        )
        
        
        
        
        
    def forward(self, x):
        
        x = self.features(x)
        x = self.classifier(x)
        
        return x
    

In [15]:
learning_rate = 0.005
epochs = 100

model = CNNModel(1).to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)




In [16]:
total_epoch_loss = []

for epoch in range(epochs):
    print(f"----------- Epoch {epoch+1}/{epochs} -----------")

    for batch_features, batch_labels in train_loader:
        
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        
        # forward pass
        outputs = model(batch_features)

        # loss calculation
        loss = loss_function(outputs, batch_labels.squeeze())
        total_epoch_loss.append(loss.item())

        # backprop

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    print(f" avg epoch loss: {sum(total_epoch_loss)/len(total_epoch_loss)} ")

----------- Epoch 1/100 -----------
 avg epoch loss: 0.561452105452617 
----------- Epoch 2/100 -----------
 avg epoch loss: 0.4752912849361698 
----------- Epoch 3/100 -----------
 avg epoch loss: 0.43199325500428676 
----------- Epoch 4/100 -----------
 avg epoch loss: 0.40502142610463004 
----------- Epoch 5/100 -----------
 avg epoch loss: 0.38390649205222727 
----------- Epoch 6/100 -----------
 avg epoch loss: 0.3676715568843194 
----------- Epoch 7/100 -----------
 avg epoch loss: 0.3542973757983141 
----------- Epoch 8/100 -----------
 avg epoch loss: 0.343201029076629 
----------- Epoch 9/100 -----------
 avg epoch loss: 0.33398337615488305 
----------- Epoch 10/100 -----------
 avg epoch loss: 0.326018210336566 
----------- Epoch 11/100 -----------
 avg epoch loss: 0.3184851471904778 
----------- Epoch 12/100 -----------
 avg epoch loss: 0.31121177931316196 
----------- Epoch 13/100 -----------
 avg epoch loss: 0.3053319476502828 
----------- Epoch 14/100 -----------
 avg epo

In [17]:
        
# evaluation

model.eval()


# return accuracy

total = 0
correct = 0 
i = 1
cnt =0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:

        cnt +=1
        
        # move data to gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            
        test_outputs = model(batch_features)

        _, predicted = torch.max(test_outputs, 1)
        
        
        
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels.squeeze()).sum().item()


accuracy = 100 * correct / total

print(accuracy) # 91.075


91.075
